In [1]:
import xarray as xr
import itertools

from py_wake.deficit_models import EddyViscosityModel


import sys
sys.path.append("..")

import src.utils as utils
import src.pywake_utils as py_wake_utils
import src.data_utils as data_utils

ModuleNotFoundError: No module named 'data_utils'

# Preparation of the parameters for the data generation

## Site (to set turbulence intensity TI)
For a given position, reference wind speed (WSref) and wind direction (WDref), Site provides the local wind condition in terms of wind speed (WS), wind direction (WD), turbulence intensity (TI) and the probability of each combination of wind direction and wind speed. Furthermore, Site is responsible for calculating the down-wind, cross-wind and vertical distance between wind turbines (which in non-flat terrain is different from the straight-line distances).

## Turbine (to set thrust coefficient CT)
For a given wind turbine type and effective wind speed (WSeff), the WindTurbine object provides the power and thrust coefficient (CT), as well as the wind turbine hub height (H) and diameter (D).

Initializing a simple WindTurbine
- taking diameter, hub height and power norm from IEA37 as these params affect the simulation;
- default power ct function to set a constant thrust coefficient (CT).

## Discretization
My current reasoning for the discretization:

FOR X RANGE
- the start is diameter*2 to skip the near-wake (careful with the wind direction, if it is within the whole range (0, 360), near-wake intercepted);
- the end is diameter*50 (i.e. DEFAULT_MAXIMUM_WAKE_DISTANCE in PyWake EddyViscosity definition); [#TODO reduce it?]
- the step is parametrized (Javier's thesis trying different values, e.g. diameter/2, diameter/4, ..., diameter/16)

FOR Y RANGE
- since the wind direction remains stable at 270, it is useless to put a big range for y

# Actual data generation

In [ ]:
# default parameters
TURBINE_X = [0]
TURBINE_Y = [0]
WIND_DIRECTION = 270  # the wind turbine's yaw angle is always adjusted according to the wind direction, thus it is probably useless to generate data with more wind directions
# WD_RANGE = range(0, 360) # (by default, 0-360° in bins of 1°)

# IEA37 values
TURBINE_DIAMETER = 198
TURBINE_HUB_HEIGHT = 119
TURBINE_POWER_NORM = 10000

# discretization factors
# (decided putting a minimum value for the deficit to delimiate the interesting zone of 1/100)
X_START_FACTOR = 2
X_END_FACTOR = 30
Y_START_FACTOR = -2
Y_END_FACTOR = 2
GRID_STEP_FACTOR = 1 / 8

# parameters for data generation
WS_RANGE = [12]  # range(10, 26)
TI_STEP = 0.01
CT_STEP = 0.01
TIs = utils.my_arange(0, 1, TI_STEP)  # the ti is percentage
CTs = utils.my_arange(
    0.1, 24 / 25, CT_STEP
)  # for ct=0, the wake field is not interesting
# according to the "AeroDyn Theory Manual", the maximum CT should be 24/25 -> see beginning of https://onlinelibrary.wiley.com/doi/full/10.1002/we.2688
# in the eddy_viscosity.py file of Pywake, it is said that ct has a range between 0.0 and 1.2 and ti > 0.0

In [ ]:
horizontal_grid = py_wake_utils.get_discretized_grid(
    TURBINE_DIAMETER,
    X_START_FACTOR,
    X_END_FACTOR,
    Y_START_FACTOR,
    Y_END_FACTOR,
    GRID_STEP_FACTOR,
)

print(
    f"Shape of the discretised grid: {len(horizontal_grid.x)}x{len(horizontal_grid.y)}"
)

for wind_speed in WS_RANGE:
    datasets = list()
    for ti, ct in itertools.product(TIs, CTs):
        print(f"\r{wind_speed=}\t{ti=}\t{ct=}", end="\r")
        site = py_wake_utils.get_site(ti=ti, ws=wind_speed)
        wind_turbine = py_wake_utils.get_wind_turbine(
            TURBINE_DIAMETER,
            TURBINE_HUB_HEIGHT,
            TURBINE_POWER_NORM,
            constant_ct=ct,
            ti=ti,
        )

        # single wake model
        ainslie_model = EddyViscosityModel(site, wind_turbine)

        ds = py_wake_utils.generate_wake_dataset(
            ainslie_model,
            wind_speed,
            WIND_DIRECTION,
            TURBINE_DIAMETER,
            TURBINE_X,
            TURBINE_Y,
            horizontal_grid,
            wind_turbine,
        )
        datasets.append(ds)

    print("\nSaving...", end="\r")
    filepath = data_utils.get_filepath(
        X_START_FACTOR,
        X_END_FACTOR,
        Y_START_FACTOR,
        Y_END_FACTOR,
        GRID_STEP_FACTOR,
        wind_speed,
        TI_STEP,
        CT_STEP,
    )
    final_ds = xr.concat(
        [d.stack(z=["x:D", "y:D", "ti", "ct"]) for d in datasets], "z"
    ).unstack("z")
    final_ds.to_netcdf(filepath)
    print(f"\rSaved data in '{filepath}'\r")

Shape of the discretised grid: 224x32
wind_speed=15	ti=1.0	ct=0.966


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


Saved data in 'data/discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01/ws_15.nc'
